# Solveur SAT pour le jeu Snake

Nous modélisons la grille de taille $N^2$ et les déplacements du snake à l'aide d'un tenseur de dimensions $N^2 \times T$, où $T$ représente l'horizon temporel sur lequel le snake doit manger les $p$ pommes.

Les coefficients du tenseur sont définis comme suit :
$$
a_{x,y,t} = \begin{cases} 
1, & \text{si la tête du snake est en position } (x,y) \text{ à l'instant } t \\
0, & \text{sinon}
\end{cases}
$$

L'instance du problème SAT est représentée sous forme d'une liste de tenseurs de dimensions $N^2 \times T$, avec les conventions suivantes :
- $1$ si la variable apparaît dans la clause,
- $-1$ si c'est son opposé,
- $0$ si la variable n'est pas présente dans la clause.

### Fonctionnalités
- **`gophersat.py`** : Ce module permet de générer un fichier `.cnf` interprétable par Gophersat, d'exécuter le solveur et de récupérer les réponses.
- **`plots.py`** : Ce module permet de visualiser et d'illustrer les résultats obtenus.

Ce notebook est dédié à la formulation des clauses SAT ainsi qu'à l'analyse et au commentaire des résultats obtenus.



In [152]:
# IMPORTS
from itertools import combinations, product
import random
from gophersat import create_sat, create_variable_index, format_clause

# MVP1 : Contraintes de déplacement
La longueur du serpent est fixe et vaut L. Il n'y a pas de pommes

In [153]:
clauses = []
N=5
T=10
L0=3
SNAKE_SIZE=3
variable=create_variable_index(N,T)

## Contrainte 1 : Unicité de la position

La tête du snake ne peut être qu'à une position $(x,y)$ à un instant $t$.

$$\forall t, \exists! (x,y) \text{ tel que } M(x,y,t) = 1$$

Cette contrainte se traduit par un nombre de clauses de l'ordre de \(O(N^4 \times T)\).

Formellement, $\forall t < T, \forall (x,y) \neq (x',y'), \neg M(x,y,t) \lor \neg M(x',y',t)$


In [154]:
def unicité_position(N: int, T: int):
    clauses_position = []
    for t in range(T):
        for (x1, y1), (x2, y2) in combinations(product(range(N), range(N)), 2):
            litteraux = [-variable((x1, y1, t)), -variable((x2, y2, t))]
            clauses_position.append(format_clause(litteraux))
    return clauses_position
clauses += unicité_position(N,T)

## Contrainte 2 : Existence de la position

À chaque instant \(t\), le snake doit être quelque part sur la grille.

$$\forall t, \exists (x,y) \text{ tel que } M(x,y,t) = 1$$

Cette contrainte garantit que la tête du snake occupe au moins une case à chaque instant \(t\).

Cela introduit \(T\) clauses :

$$\forall t, \bigvee_{x,y} M(x,y,t)$$



In [155]:
def existence_position(N: int, T: int):
    clauses_position = []
    for t in range(T):
        litteraux = [variable((x, y, t)) for x, y in product(range(N), range(N))]
        clauses_position.append(format_clause(litteraux))
    return clauses_position


clauses += existence_position(N,T)

## Contrainte 3 : Déplacement valide

La position en \(t+1\) doit être adjacente à la position en \(t\).

$$\forall t, \forall (x,y), M(x,y,t) \Rightarrow M(x,y-1,t+1) \lor M(x,y+1,t+1) \lor M(x-1,y,t+1) \lor M(x+1,y,t+1)$$

Cela se traduit par $O(N^2 \times T)$ clauses supplémentaires.

Formellement :

$$\forall x,y,t, \neg M(x,y,t) \lor M(x,y-1,t+1) \lor M(x,y+1,t+1) \lor M(x-1,y,t+1) \lor M(x+1,y,t+1)$$



In [156]:
def positions_adjacentes(x,y,N):
    x_adjacentes = []
    y_adjacentes = []
    if x != N-1:
        x_adjacentes.append(x+1)
    if x != 0:
        x_adjacentes.append(x-1)
    if y != N-1:
        y_adjacentes.append(y+1)
    if y != 0:
        y_adjacentes.append(y-1)
    return [(x,y2) for y2 in y_adjacentes] + [(x2,y) for x2 in x_adjacentes]

def next_move(N: int, T: int):
    clauses_position = []
    for x, y, t in product(range(N), range(N), range(T - 1)):
        litteraux = [-variable((x, y, t))] + [variable((x2, y2, t + 1)) for x2, y2 in positions_adjacentes(x, y, N)]
        clauses_position.append(format_clause(litteraux))
    return clauses_position

clauses += next_move(N, T)

## Contrainte 4 : Ne pas se mordre la queue

Pour l'instant, le serpent a une longueur fixe \(L\). La position de la tête du snake \((x,y)\) sur la grille ne peut donc pas être la même à un instant \(t\) et à un instant \(t'\) avec \(t' < t + L\). Cela se traduit par l'introduction de $O(N^2 \times L \times T)$ clauses supplémentaires.

$$\forall t, \forall l < L, \forall (x,y), \neg M(x,y,t) \lor \neg M(x,y,t+l)$$





In [157]:
def mordre_queue_v1(N: int, T: int, L: int):
    clauses_position = []
    for x, y, t in product(range(N), range(N), range(T - 1)):
        for l in range(1, min(T - t, L)):
            litteraux=[-variable((x, y, t)), -variable((x, y, t + l))]
            clauses_position.append(format_clause(litteraux))
    return clauses_position

clauses += mordre_queue_v1(N,T,L0)

## Résultats MVP1

In [158]:
solution = create_sat(clauses)
import matplotlib.pyplot as plt
from plots import create_snake_path, visualize_snake
snake_path = create_snake_path(solution, T, N, variable)
visualize_snake(N, snake_path, animation_name="solution_animation_MVP1.gif")
plt.close()

# MVP 2 : Pommes et croissance du snake
Cette fois ci on place p pommes sur la grille. Les positions sont dans la liste pommes. Le snake doit avoir mangée toutes ses pommes avant la fin du jeu, c'est à dire avant t=T. De plus le snake grandit de une unité à chaque pomme mangée.


In [159]:
clauses = []
N=10
T=35
L=3
nb_pommes = 9


pommes = random.sample(list(product(range(N), range(N))), nb_pommes)        #Pommes placées aléatoirement sur la grille
variable=create_variable_index(N,T, nb_pommes)

## Contrainte 5 : Initialiser les pommes

Au début du jeu, toutes les pommes sont actives
$$\forall (x,y) \in pommes, P(x,y,0)$$

Cela se traduit par $O(P)$ contraintes supplémentaires.

In [160]:
def init_pommes(nb_pommes):
    clauses_position = []
    for p in range(nb_pommes):
        litteraux=[variable((p, 0), 'p')]
        clauses_position.append(format_clause(litteraux))
    return clauses_position

## Contrainte 6 : Initialiser la position hors des pommes

Au début du jeu, le serpent n'est pas sur une pomme
$$\forall (x,y) \in pommes, \neg M(x,y,0)$$

Cela se traduit par $O(P)$ contraintes supplémentaires.

In [161]:
def init_positions(pommes):
    clauses_position = []
    for (x,y) in pommes:
        litteraux=[-variable((x,y, 0), 's')]
        clauses_position.append(format_clause(litteraux))
    return clauses_position

## Contrainte 7 : Faire disparaitre les pommes

A chaque fois que le snake passe au-dessus d'une pomme "active", elle disparait.
$$\forall t, \forall (x,y) \in pommes, M(x,y,t) \land P(x,y,t) \Rightarrow \neg P(x,y,t+1)$$

Cela se traduit par $O(P \times T)$ contraintes supplémentaires. Formellement:
$$\forall t, \forall (x,y) \in pommes, \neg M(x,y,t) \lor \neg P(x,y,t) \lor \neg P(x,y,t+1)$$

In [162]:
def disparaitre_pommes(pommes, T):
    clauses_position = []
    for p, (x,y) in enumerate(pommes):
        for t in range(T-1):
            litteraux=[-variable((x, y, t), 's'), -variable((p, t), 'p'), -variable((p, t+1), 'p')]
            clauses_position.append(format_clause(litteraux))
    return clauses_position

## Contrainte 8 : Pas d'anihilation spontanée de pommes

Les pommes restent "actives" si le snake ne passe pas dessus
$$\forall t, \forall (x,y) \in pommes, P(x,y,t) \Rightarrow P(x,y,t+1) \lor M(x,y,t)$$

Cela se traduit par $O(P \times T)$ contraintes supplémentaires. Formellement:
$$\forall t, \forall (x,y) \in pommes, \neg P(x,y,t) \lor P(x,y,t+1) \lor M(x,y,t)$$

In [163]:
def preserver_pommes(pommes, T):
    clauses_position = []
    for p, (x,y) in enumerate(pommes):
        for t in range(T-1):
            litteraux=[-variable((p, t), 'p'), variable((p, t+1), 'p'), variable((x, y, t), 's')]
            clauses_position.append(format_clause(litteraux))
    return clauses_position

## Contrainte 9 : Pas de création spontanée de pommes

Les pommes restent "inactives" une fois mangées
$$\forall t, \forall (x,y) \in pommes, \neg P(x,y,t) \Rightarrow \neg P(x,y,t+1)$$

Cela se traduit par $O(P \times T)$ contraintes supplémentaires. Formellement:
$$\forall t, \forall (x,y) \in pommes, P(x,y,t) \lor \neg P(x,y,t+1)$$

In [164]:
def ne_pas_creer_pommes(nb_pommes, T):
    clauses_position = []
    for p in range(nb_pommes):
        for t in range(T-1):
            litteraux=[variable((p, t), 'p'), -variable((p, t+1), 'p')]
            clauses_position.append(format_clause(litteraux))
    return clauses_position

## Deux implémentations possibles

Le snake grandit d'une case à chaque pomme mangée. L'implémentation de cette contrainte nécessite l'ajout de clauses et, potentiellement, de variables supplémentaires. Deux approches sont explorées ici.

### Première approche : Utilisation d'un compteur de pommes mangées

Précédemment, lorsque la longueur du snake était fixe, la contrainte "ne pas se mordre la queue" se traduisait par :
$$
\forall (x,y,t), \forall l < L, \neg M(x,y,t) \lor \neg M(x,y,t+l)
$$

Désormais, cette contrainte doit être implémentée dynamiquement pour chaque longueur du snake \(L\), qui dépend du nombre de pommes mangées. On peut introduire une variable compteur $p_{mangée}$ pour suivre ce nombre.

Le nombre de clauses associé à..... la contrainte "ne pas se mordre la queue" (dans le MVP1 $O(N^2 \times L \times T)$) devient :
$$
\sum_{p_{mangée}=0}^{n_{pommes}} N^2 \times (L_0 + p_{mangée}) \times T = N^2 \times T \times \left( n_{pommes} L_0 + \frac{n_{pommes} (n_{pommes}+1)}{2} \right)
$$

À cela s'ajoutent les clauses qui implémentent l'évolution du compteur de pommes mangées.

### Seconde approche : Déduction du nombre de pommes mangées

Une autre possibilité est de déduire $p_{mangée}$ directement à partir de la matrice de variables indiquant la présence des pommes actives. Dans ce cas, il faut prendre en compte, pour chaque p_{mangée}, toutes les combinaisons possibles de pommes mangées. Cela correspond donc à :
$$
\sum_{p_{mangée}=0}^{n_{pommes}} \binom{n_{pommes}}{p_{mangée}} \times N^2 \times (L_0 + p) \times T
$$

Développons cette somme :
$$
N^2 \times T \times \sum_{p=0}^{n_{pommes}} \binom{n_{pommes}}{p} (L_0 + p)
$$
En utilisant la propriété de la somme des coefficients binomiaux :
$\sum_{p=0}^{n_{pommes}} \binom{n_{pommes}}{p} = 2^{n_{pommes}}$ et $\sum_{p=0}^{n_{pommes}} \binom{n_{pommes}}{p}\times p = n_{pommes} \times 2^{n_{pommes}-1}$


On obtient finalement :
$$
N^2 \times T \times \left( 2^{n_{pommes}} L_0 + n_{pommes} \cdot 2^{n_{pommes}-1} \right)
$$

Cette seconde approche introduit un facteur exponentiel $2^{n_{pommes}}$, ce qui peut rendre l'implémentation plus coûteuse en termes de complexité. Cependant, on n'ajoute pas de variables supplémentaires.



## Approche 1. Ajout de variables "counter"

### Contrainte 10.a : Comptage des pommes

Une pomme qui est mangée incrémente le compteur
$$\forall t, \forall c, \forall (x,y) \in pommes, M(x,y,t) \land P(x,y,t) \land C(c,t) \land \neg C(c+1,t) \Rightarrow C(c+1,t+1)$$

Cela se traduit par $O(P \times T \times C)$ contraintes supplémentaires. Formellement:
$$\forall t, \forall c, \forall (x,y) \in pommes, \neg M(x,y,t) \lor \neg P(x,y,t) \lor \neg C(c,t) \lor C(c+1,t) \lor C(c+1,t+1)$$

In [165]:
def comptage_pommes(pommes, T):
    clauses_position = []
    for p, (x,y) in enumerate(pommes):
        for t in range(T):
            for c in range(nb_pommes-1):
                litteraux=[-variable((x, y, t), 's'), -variable((p, t), 'p'), -variable((c, t), 'c'), variable((c+1, t), 'c'), variable((c+1, t+1), 'c')]
                clauses_position.append(format_clause(litteraux))
    return clauses_position

### Contrainte 10.b : Initialisation du compteur

Le compteur est initialement à 0 et la première pomme mangée le démarre
$$\forall c, \neg C(c,0)$$
$$\forall t, \forall (x,y) \in pommes, M(x,y,t) \land P(x,y,t) \Rightarrow C(0,t+1)$$

Cela se traduit par $O(P \times T)$ contraintes supplémentaires. Formellement:
$$\forall c, \neg C(c,0)$$
$$\forall t, \forall (x,y) \in pommes, \neg M(x,y,t) \lor \neg P(x,y,t) \lor C(0,t+1)$$

In [166]:
def init_compteur(pommes, T):
    clauses_position = []
    
    for c in range(len(pommes)):
        litteraux=[-variable((c, 0), 'c')]
        clauses_position.append(format_clause(litteraux))


    for p, (x,y) in enumerate(pommes):
        for t in range(T):
            litteraux=[-variable((x, y, t), 's'), -variable((p, t), 'p'), variable((0, t+1), 'c')]
            clauses_position.append(format_clause(litteraux))
    return clauses_position

### Contrainte 10.c : Pas de désincrémentation du compteur

Toute incrémentation du compteur est définitive
$$\forall t, \forall c, C(c,t) \Rightarrow C(c,t+1)$$

Cela se traduit par $O(T \times C)$ contraintes supplémentaires. Formellement:
$$\forall t, \forall c, \neg C(c,t) \lor C(c,t+1)$$

In [167]:
def ne_pas_desincrementer_compteur(nb_pommes, T):
    clauses_position = []
    for t in range(T):
        for c in range(nb_pommes):
            litteraux=[-variable((c, t), 'c'), variable((c, t+1), 'c')]
            clauses_position.append(format_clause(litteraux))
    return clauses_position

### Contrainte 10.d : Pas d'incrémentation spontanée du compteur

Le compteur reste fixe si aucune pomme n'est mangée
$$\forall t, \forall c, \neg C(c, t) \land C(c,t+1) \Rightarrow \bigvee_{x,y \in pommes} (M(x,y,t) \land P(x,y,t))$$

Sans perte de généralité, on peut réduire la complexité de ce problème en:
$$\forall t, \forall c, \neg C(c, t) \land C(c,t+1) \Rightarrow \bigvee_{x,y \in pommes} M(x,y,t)$$
$$\forall t, \forall c, \forall (x,y) \in pommes, \neg C(c, t) \land C(c,t+1) \land M(x,y,t) \Rightarrow P(x,y,t)$$

Formellement, cela se traduit par:
$$\forall t, \forall c, C(c, t) \lor \neg C(c,t+1) \lor \bigvee_{x,y \in pommes} M(x,y,t)$$
$$\forall t, \forall c, \forall (x,y) \in pommes, C(c, t) \lor \neg C(c,t+1) \lor \neg M(x,y,t) \lor P(x,y,t)$$

In [168]:
def ne_pas_incrementer_compteur(nb_pommes, T, pommes):
    clauses_position = []
    for t in range(T):
        for c in range(nb_pommes):
            litteraux1=[variable((c, t), 'c'), -variable((c, t+1), 'c')]
            for p, (x,y) in enumerate(pommes):
                litteraux1.append(variable((x, y, t), 's'))
                litteraux2 = [variable((c, t), 'c'), -variable((c, t+1), 'c'), -variable((x, y, t), 's'), variable((p, t), 'p')]
                clauses_position.append(format_clause(litteraux2))

            clauses_position.append(format_clause(litteraux1))

    return clauses_position

### Contrainte 10.e : Incrémentation du compteur d'un seul incrément à la fois

L'incrémentation du compteur se fait un incrément à la fois
$$\forall t, \forall (c, c'), c' > c , \neg C(c,t) \land C(c,t+1) \Rightarrow \neg C(c',t+1)$$

Cela se traduit par $O(T \times C^2)$ contraintes supplémentaires. Formellement:
$$\forall t, \forall (c, c'), c' > c , C(c,t) \lor \neg C(c,t+1) \lor \neg C(c',t+1)$$

In [169]:
def increment_unique(nb_pommes, T):
    clauses_position = []
    for t in range(T):
        for c in range(nb_pommes):
            for c2 in range(c+1, nb_pommes):
                litteraux=[variable((c, t), 'c'), -variable((c, t+1), 'c'), -variable((c2, t+1), 'c')]
                clauses_position.append(format_clause(litteraux))
    return clauses_position

### Contrainte 10.f : Manger toutes les pommes

Toutes les pommes doivent être mangées à la fin du temps imparti. Cela se traduit par la contrainte
$$C(P-1,T)$$

In [170]:
def manger_des_pommes(nb_pommes, T):
    litteral = [variable((nb_pommes - 1, T), 'c')]
    clauses_position = [format_clause(litteral)]
    return clauses_position

### Contrainte 10.g : Ne pas se mordre la queue V2

Le serpent a désormais une longueur variable, incrémentée de 1 à chaque fois que le serpent mange une pomme. La position de la tête du snake \((x,y)\) sur la grille ne peut donc pas être la même à un instant \(t\) et à un instant \(t'\) avec \(t' < t + L + nb_pommes_mangees\). Cela se traduit par l'introduction de $O(N^2 \times (L+P) \times T)$ clauses supplémentaires.

$$\forall t, \forall l < L, \forall (x,y), \neg M(x,y,t) \lor \neg M(x,y,t+l)$$
$$\forall t, \forall 0 \leq c < P, \forall (x,y), C(c,t) \Rightarrow (\neg M(x,y,t) \lor \neg M(x,y,t+L+c))$$

Soit formellement:
$$\forall t, \forall l < L, \forall (x,y), \neg M(x,y,t) \lor \neg M(x,y,t+l)$$
$$\forall t, \forall 0 \leq c < P, \forall (x,y), \neg C(c,t) \lor \neg M(x,y,t) \lor \neg M(x,y,t+L+c)$$

In [171]:
def mordre_queue_v2(N: int, T: int, L: int, nb_pommes: int):
    clauses_position = []
    for x, y, t in product(range(N), range(N), range(T - 1)):
        for l in range(1, min(T - t, L)):
            litteraux=[-variable((x, y, t)), -variable((x, y, t + l))]
            clauses_position.append(format_clause(litteraux))

        for c in range(min(T -L - t, nb_pommes)):
            litteraux=[-variable((c,t), 'c') ,-variable((x, y, t)), -variable((x, y, t + L + c))]
            clauses_position.append(format_clause(litteraux))
    return clauses_position

## Resultats approche 1

In [172]:
clauses = []
clauses += unicité_position(N,T)
clauses += existence_position(N,T)
clauses += next_move(N,T)
clauses += init_pommes(nb_pommes)
clauses += init_positions(pommes)
clauses += disparaitre_pommes(pommes, T)
clauses += preserver_pommes(pommes, T)
clauses += ne_pas_creer_pommes(nb_pommes, T)

In [173]:
clauses += comptage_pommes(pommes, T)
clauses += init_compteur(pommes, T)
clauses += ne_pas_desincrementer_compteur(nb_pommes, T)
clauses += ne_pas_incrementer_compteur(nb_pommes, T, pommes)
clauses += increment_unique(nb_pommes, T)
clauses += manger_des_pommes(nb_pommes, T)
clauses += mordre_queue_v2(N,T,L0, nb_pommes)

In [174]:
solution = create_sat(clauses)
assert len(solution) > 0
snake_path = create_snake_path(solution, T, N, variable, pommes=pommes)
visualize_snake(N, snake_path, animation_name="solution_animation_MVP2A.gif", apple_positions=pommes, interval=400)
plt.close()

## Approche 2. Ajout de davantage de clauses

### Contrainte 11.a : Ne pas se mordre la queue

Pour chaque nombre de pommes mangées $p_{mangées}$, il existe $\binom{n_{pommes}}{p_{mangées}}$ combinaisons possibles $(p_i)_{0<i\leq p_{mangées}}$.

Si ces pommes ont été mangées à l'instant t+l, (c'est-à-dire qu'elles ne sont plus actives), on note $ \bigwedge_{i=1}^{p_{mangées}} \neg P(i,t+l)$

Alors la condition suivante doit être respectée :

$$
\forall l < L_0 + p_{mangées}, \neg M(x,y,t) \lor \neg M(x,y,t+l)
$$

Ce qui peut être réécrit sous forme équivalente :

$$
\forall (x,y,t), \forall l < L_0 + p_{mangées}, \left( \bigvee_{i=1}^{p_{mangées}} P(i,t+l) \right) \lor \neg M(x,y,t) \lor \neg M(x,y,t+l)
$$

Cette contrainte doit être appliquée pour tout $p_{mangées}$ et pour chaque combinaison $(p_i)_{0<i\leq p_{mangées}}$.



In [175]:
def mordre_queue_v3(N: int, T: int, L0: int, nb_pommes:int):
    clauses_position = []
    for x, y, t, in product(range(N), range(N), range(T - 1)):
        for p_mangee in range(t+1):
            for selected_pommes in combinations(range(nb_pommes), p_mangee):
                for l in range(1, min(T - t, L0+p_mangee)):
                    litteraux=[-variable((x, y, t)), -variable((x, y, t + l))]
                    litteraux += [variable(typ='p', id=(pomme_idx, t+l)) for pomme_idx in selected_pommes]
                    clauses_position.append(format_clause(litteraux))
    return clauses_position

### Contrainte 11.b : Mangez des pommes
Toutes les pommes doivent être mangées à la fin du jeu donc à t=T-1, elles ne sont plus actives.
$$\forall i<n_{pommes}, \neg P(i,T-1)$$

In [176]:
def mangez_des_pommes_v2(T: int, nb_pommes:int):
    clauses_position = []
    for i in range(nb_pommes):
        litteraux=[-variable(typ='p', id=(i,T-1))]
        clauses_position.append(format_clause(litteraux))
    return clauses_position

## Résultats approche 2

In [177]:
clauses = []
clauses += unicité_position(N,T)
clauses += existence_position(N,T)
clauses += next_move(N,T)
clauses += init_pommes(nb_pommes)
clauses += init_positions(pommes)
clauses += disparaitre_pommes(pommes, T)
clauses += preserver_pommes(pommes, T)
clauses += ne_pas_creer_pommes(nb_pommes, T)

In [178]:
clauses += mangez_des_pommes_v2(T,nb_pommes)
clauses += mordre_queue_v3(N,T,L0,nb_pommes)

In [179]:
solution = create_sat(clauses)
assert len(solution) > 0
snake_path = create_snake_path(solution, T, N, variable, pommes=pommes)
visualize_snake(N, snake_path, animation_name="solution_animation_MVP2B.gif", apple_positions=pommes)
plt.close()

# MVP 3 : Dégustation des pommes dans l'ordre
Cette fois ci on considère que les pommes apparaissent dans un ordre prédéfini, connu à l'avance. Le snake devra manger chaque pomme pour révéler la suivante et les avoir toutes dégustées. Nous retiendrons dans la suite l'approche par compteur précédemment déployée. Seulement, il convient de notifier que dans le cadre de ce problème où les pommes sont ordonnées, le compteur vérifie:

$$\forall c, \forall t, C(c,t) = \neg P(c,t)$$

Par conséquent, dans un souci de simplification des contraintes, Nous supprimerons les variables associées au compteur dans cette partie

In [216]:
clauses = []
N=10
T=70
L=3
nb_pommes = 9


pommes = random.sample(list(product(range(N), range(N))), nb_pommes)        #Pommes placées aléatoirement sur la grille
variable=create_variable_index(N,T, nb_pommes)

## Contrainte 12 : Initialiser la position hors de la première pomme

Au début du jeu, le serpent n'est pas sur la pomme visible
$$\neg M(x_0,y_0,0)$$

Cela se traduit par $1$ contrainte supplémentaire.

In [217]:
def init_positions_v2(pommes):
    x0,y0 = pommes[0]
    litteral = [-variable((x0, y0, 0), 's')]
    clauses_position = [format_clause(litteral)]
    return clauses_position

## Contrainte 13 : Faire disparaitre les pommes

A chaque fois que le snake passe au-dessus de la pomme "active", elle disparait.
$$\forall t, \forall p \in pommes, M(x_p,y_p,t) \land P(p,t) \land \neg P(p-1,t) \Rightarrow \neg P(p,t+1)$$
$$\forall t, M(x_0,y_0,t) \land P(0,t) \Rightarrow \neg P(0,t+1)$$

Cela se traduit par $O(P \times T)$ contraintes supplémentaires. Formellement:
$$\forall t, \forall p \in pommes, \neg M(x_p,y_p,t) \lor \neg P(p,t) \lor P(p-1,t) \lor \neg P(p,t+1)$$
$$\forall t, \neg M(x_0,y_0,t) \lor \neg P(0,t) \lor \neg P(0,t+1)$$

In [218]:
def disparaitre_pommes_v2(pommes, T):
    clauses_position = []

    x0, y0 = pommes[0]
    for t in range(T):
        litteraux=[-variable((x0, y0, t), 's'), -variable((0, t), 'p'), -variable((0, t+1), 'p')]
        clauses_position.append(format_clause(litteraux))

        for p, (xp,yp) in enumerate(pommes):
            if p == 0:
                continue
            litteraux=[-variable((xp, yp, t), 's'), -variable((p, t), 'p'), -variable((p-1, t), 'p'), -variable((p, t+1), 'p')]
            clauses_position.append(format_clause(litteraux))
    return clauses_position

## Contrainte 14 : Pas d'anihilation spontanée de pommes

Les pommes restent "actives" si le snake ne passe pas dessus ou qu'elles ne sont pas mangeables
$$\forall t, \forall p \in pommes, P(p,t) \Rightarrow P(p,t+1) \lor (M(x_p,y_p,t) \land \neg P(p-1,t))$$
$$\forall t, P(0,t) \Rightarrow P(0,t+1) \lor M(x_0,y_0,t)$$

Cela se traduit par $O(P \times T)$ contraintes supplémentaires. Formellement:
$$\forall t, \forall p \in pommes, \neg P(p,t) \lor P(p,t+1) \lor M(x_p,y_p,t)$$
$$\forall t, \forall p \in pommes, \neg P(p,t) \lor P(p,t+1) \lor \neg P(p-1,t)$$

In [219]:
def preserver_pommes_v2(pommes, T):
    clauses_position = []
    for t in range(T):
        x0, y0 = pommes[0]
        litteraux=[-variable((0, t), 'p'), variable((0, t+1), 'p'), variable((x0, y0, t), 's')]
        clauses_position.append(format_clause(litteraux))

        for p, (xp,yp) in enumerate(pommes):
            if p == 0:
                continue
            litteraux=[-variable((p, t), 'p'), variable((p, t+1), 'p'), variable((xp, yp, t), 's')]
            clauses_position.append(format_clause(litteraux))

            litteraux=[-variable((p, t), 'p'), variable((p, t+1), 'p'), -variable((p-1, t), 'p')]
            clauses_position.append(format_clause(litteraux))
    return clauses_position

## Contrainte 15 : Manger toutes les pommes

Toutes les pommes doivent être mangées à la fin du temps imparti. Cela se traduit par la contrainte
$$\neg P(P-1,T)$$

In [220]:
def manger_des_pommes_v2(nb_pommes, T):
    litteral = [-variable((nb_pommes - 1, T), 'p')]
    clauses_position = [format_clause(litteral)]
    return clauses_position

## Contrainte 16 : Ne pas se mordre la queue V2

Le serpent a désormais une longueur variable, incrémentée de 1 à chaque fois que le serpent mange une pomme. La position de la tête du snake \((x,y)\) sur la grille ne peut donc pas être la même à un instant \(t\) et à un instant \(t'\) avec \(t' < t + L + nb_pommes_mangees\). Cela se traduit par l'introduction de $O(N^2 \times (L+P) \times T)$ clauses supplémentaires.

$$\forall t, \forall l < L, \forall (x,y), \neg M(x,y,t) \lor \neg M(x,y,t+l)$$
$$\forall t, \forall 0 \leq p < P, \forall (x,y), \neg P(p,t) \Rightarrow (\neg M(x,y,t) \lor \neg M(x,y,t+L+p))$$

Soit formellement:
$$\forall t, \forall l < L, \forall (x,y), \neg M(x,y,t) \lor \neg M(x,y,t+l)$$
$$\forall t, \forall 0 \leq p < P, \forall (x,y), P(p,t) \lor \neg M(x,y,t) \lor \neg M(x,y,t+L+p)$$

In [221]:
def mordre_queue_v4(N: int, T: int, L: int, nb_pommes: int):
    clauses_position = []
    for x, y, t in product(range(N), range(N), range(T - 1)):
        for l in range(1, min(T - t, L)):
            litteraux=[-variable((x, y, t)), -variable((x, y, t + l))]
            clauses_position.append(format_clause(litteraux))

        for p in range(min(T -L - t, nb_pommes)):
            litteraux=[variable((p,t), 'p') ,-variable((x, y, t)), -variable((x, y, t + L + p))]
            clauses_position.append(format_clause(litteraux))
    return clauses_position

## Resultats MVP3

In [222]:
clauses = []
clauses += unicité_position(N,T)
clauses += existence_position(N,T)
clauses += next_move(N,T)
clauses += init_pommes(nb_pommes)
clauses += init_positions_v2(pommes)
clauses += disparaitre_pommes_v2(pommes, T)
clauses += preserver_pommes_v2(pommes, T)
clauses += ne_pas_creer_pommes(nb_pommes, T)

In [223]:
clauses += manger_des_pommes_v2(nb_pommes, T)
clauses += mordre_queue_v4(N,T,L0, nb_pommes)

In [224]:
solution = create_sat(clauses)
assert len(solution) > 0
snake_path = create_snake_path(solution, T, N, variable, pommes=pommes)
visualize_snake(N, snake_path, animation_name="solution_animation_MVP3.gif", apple_positions=pommes, pommes_successives=True, interval=400)
plt.close()